message with type additional steps
force use tool


have a state
agent = Agent()

agent()

In [104]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [105]:
import json

In [106]:
from langchain.agents import create_openai_functions_agent, AgentExecutor, tool
from langchain_openai import ChatOpenAI
from typing import Literal, Optional
from pydantic import BaseModel, Field

class Address(BaseModel):
    AddressType: Optional[Literal['POBOX', 'STREET']] = Field(
        None, description="The type of address, POBOX for mailing addresses or STREET for physical addresses"
    )
    AddressLine1: Optional[str] = Field(
        None, description="First line of the address (e.g., street address or PO box)"
    )
    City: Optional[str] = Field(
        None, description="The city for the address"
    )
    Region: Optional[str] = Field(
        None, description="The region or state for the address"
    )
    PostalCode: Optional[str] = Field(
        None, description="The postal or ZIP code for the address"
    )
    Country: Optional[str] = Field(
        None, description="The country, using only alphabetic characters (A-Z, a-z)"
    )

class ContactLLM(BaseModel):
    Name: str = Field(description="Name of the business or individual.")
    Addresses: Optional[list[Address]] = Field(None, description="Addresses Info of the business or individual")
    EmailAddress: Optional[str] = Field(None, description="Email of the business or individual")


class LineItemLLM(BaseModel):
    Description: str = Field(description="A line item with just a description (i.e no unit amount or quantity) can be created by specifying just a Description element that contains at least 1 character")
    UnitAmount: Optional[str] = Field(default=None, description="Unit price or rate per item, it may be empty if price is listed as line_amount")
    LineAmount: Optional[str] = Field(default=None, description="Total item price, it may be empty if price is listed as line_amount")
    Quantity: int = Field(default=1, description="Quantity of the item.")
    AccountCode: str = Field(description="This field is related to xero, the account code of expense category")
    AccountId: str = Field(description="This field is related to xero, the account id of expense category")
    IsTaxable: bool = Field(description="whether item is subject to tax, Item is taxable if there is tax_rate or total tax and not part of additional fees ie admin fee etc")

class ReceiptLLM(BaseModel):
    # receipt_from: Optional[str] = Field(description="The provider of the receipt")
    Contact: ContactLLM = Field(description="ContaTaxRatesct info of the business or individual.")
    Date: str = Field(description="Date of receipt – YYYY-MM-DD")
    Reference: Optional[str] = Field(description="Additional reference number")
    
    # highlight this in prompt
    LineAmountTypes: Literal["Exclusive", "Inclusive"] = Field(description="The type of tax in the receipt")
    
    SubTotal: str = Field(description="Total of receipt excluding taxes")
    TotalTax: Optional[str] = Field(default=None, description="Total tax on receipt")
    
    Total: str = Field(description="Total of receipt tax inclusive (i.e. SubTotal + TotalTax)")
    
    LineItems: list[LineItemLLM] = Field(description="List of line item included in receipt")
    TaxRate: Optional[float] = Field(None, description="The tax rate of receipt in percentage")
    TaxType: Literal["VAT", "GST"] = Field(description="The type of tax given on the receipt, refer to the given category/account type to determine correct tax type")



In [107]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage

prompt = ChatPromptTemplate(
    [
        ("system", (
            "Your are helpful question and answer assistant"
            "Answer the following question based on this context\n\n"
            "Context: {context}"
            "Keep your answer concise"
            "You are equip with tools, make sure to route the query to appropriate tool if needed"
            "If given empty query, analyze the context and see if theres tool related to it and use it\n\n"

            "Instructions for Receipt Specific Query: "
            "1. If user submits an empty query, analyze the receipt and respond short summary of the receipt"
            "2. Format the output message in this ff format:"
                """
                    Here’s a summary of the important details from your [name of bill] receipt. This breakdown highlights the key charges and payment information.

                    [display here the very short summary of receipt, keep it 8 bullet points at most, list items purchased in bullet points, FInally compact the list]
                    
                    To help categorize this receipt properly, please choose a Category from the dropdown below.
                """
            "\n\n 3. Wait for the user to enter the category for the receipt, only save the receipt in xero when category/account is given by user"
            "4. 'Category' or 'Account' will be treated as the same" 
            "5. If account/category is given by user, automatically save the receipt to xero"
        )),
        
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [108]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import HumanMessage, AIMessage

@tool
def save_receipt(receipt: ReceiptLLM, category: str):
    """This tool accepts receipt and category then save to xero management app"""

    print("Saving")

In [109]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
tools = [save_receipt]
agent = create_openai_functions_agent(llm=model, tools=tools, prompt=prompt)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chat_history = []
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [110]:
context_json = {
  "account": "string",
  "receipt": {
    "contact": {
      "name": "The Tack Room",
      "addresses": [
        {
          "address_type": "STREET",
          "address_line_1": "145 Lincoln Road",
          "address_line_2": None,
          "address_line_3": None,
          "address_line_4": None,
          "city": "Lincoln",
          "region": "MA",
          "postal_code": "01773",
          "country": "US"
        }
      ],
      "email_address": None
    },
    "date": "2024-04-08",
    "reference": "Check #36",
    "line_amount_types": "Exclusive",
    "subtotal": "114.00",
    "total_tax": "7.11",
    "total": "124.53",
    "line_items": [
      {
        "description": "BBQ Potato Chips",
        "unit_amount": None,
        "line_amount": "7.00",
        "quantity": 1,
        "is_taxable": True
      },
      {
        "description": "Diet Coke",
        "unit_amount": None,
        "line_amount": "3.00",
        "quantity": 1,
        "is_taxable": True
      },
      {
        "description": "Trillium Fort Point",
        "unit_amount": None,
        "line_amount": "10.00",
        "quantity": 1,
        "is_taxable": True
      },
      {
        "description": "Fried Chicken Sandwich",
        "unit_amount": None,
        "line_amount": "34.00",
        "quantity": 2,
        "is_taxable": True
      },
      {
        "description": "Famous Duck Grilled Cheese",
        "unit_amount": None,
        "line_amount": "25.00",
        "quantity": 1,
        "is_taxable": True
      },
      {
        "description": "Mac & Cheese",
        "unit_amount": None,
        "line_amount": "17.00",
        "quantity": 1,
        "is_taxable": True
      },
      {
        "description": "Burger of the moment",
        "unit_amount": None,
        "line_amount": "18.00",
        "quantity": 1,
        "is_taxable": True
      },
      {
        "description": "Admin Fee",
        "unit_amount": None,
        "line_amount": "3.42",
        "quantity": 1,
        "is_taxable": False
      }
    ],
    "tax_rate": 6.24
  }
}

query = ""
file_id = "aca5c01e-b490-4c86-9c80-bd0179b3f021"
chat_history.append(HumanMessage(content=query, files=[file_id]))
result = executor.invoke({
    "input": "",
    "context": json.dumps(context_json),
    "chat_history": chat_history
})

chat_history.append(AIMessage(result["output"]))





> Entering new AgentExecutor chain...
Here’s a summary of the important details from your receipt. This breakdown highlights the key charges and payment information.

- **Date:** April 8, 2024
- **Reference:** Check #36
- **Subtotal:** $114.00
- **Total Tax:** $7.11
- **Total Amount:** $124.53
- **Items Purchased:**
  - BBQ Potato Chips - $7.00
  - Diet Coke - $3.00
  - Trillium Fort Point - $10.00
  - Fried Chicken Sandwich (2) - $68.00
  - Famous Duck Grilled Cheese - $25.00
  - Mac & Cheese - $17.00
  - Burger of the moment - $18.00
  - Admin Fee - $3.42

To help categorize this receipt properly, please choose a Category from the dropdown below.

> Finished chain.


In [111]:
query = "402 - Entertainment"
result = executor.invoke({
    "input": query,
    "context": json.dumps(context_json),
    "chat_history": chat_history
})
chat_history.append(HumanMessage(content=query, parent_id="001"))
chat_history.append(AIMessage(content=result["output"]))



> Entering new AgentExecutor chain...

Invoking: `save_receipt` with `{'receipt': {'Contact': {'Name': 'The Tack Room', 'Addresses': [{'AddressType': 'STREET', 'AddressLine1': '145 Lincoln Road', 'City': 'Lincoln', 'Region': 'MA', 'PostalCode': '01773', 'Country': 'US'}], 'EmailAddress': None}, 'Date': '2024-04-08', 'Reference': 'Check #36', 'LineAmountTypes': 'Exclusive', 'SubTotal': '114.00', 'TotalTax': '7.11', 'Total': '124.53', 'LineItems': [{'Description': 'BBQ Potato Chips', 'LineAmount': '7.00', 'Quantity': 1, 'IsTaxable': True, 'AccountCode': '402', 'AccountId': '1'}, {'Description': 'Diet Coke', 'LineAmount': '3.00', 'Quantity': 1, 'IsTaxable': True, 'AccountCode': '402', 'AccountId': '1'}, {'Description': 'Trillium Fort Point', 'LineAmount': '10.00', 'Quantity': 1, 'IsTaxable': True, 'AccountCode': '402', 'AccountId': '1'}, {'Description': 'Fried Chicken Sandwich', 'LineAmount': '34.00', 'Quantity': 2, 'IsTaxable': True, 'AccountCode': '402', 'AccountId': '1'}, {'Descripti